# **Table Design**
## RAW TABLE


In [20]:
import pandas as pd
raw_table = pd.DataFrame([['DATETIME', 'VARCHAR2(30 BYTE)', 'N', 'Yes', 'null', 1, 'Data from raw file'],
                          ['VIDEOTITLE', 'VARCHAR2(200 BYTE)', 'N', 'Yes', 'null', 2, 'Data from raw file'],
                          ['EVENTS', 'VARCHAR2(150 BYTE)', 'N', 'Yes', 'null', 3, 'Data from raw file']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])

raw_table.style.hide_index().set_caption('RAW TABLE')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME,VARCHAR2(30 BYTE),N,Yes,null,1,Data from raw file
VIDEOTITLE,VARCHAR2(200 BYTE),N,Yes,null,2,Data from raw file
EVENTS,VARCHAR2(150 BYTE),N,Yes,null,3,Data from raw file


## Dimension Table DIMDATE 

In [12]:
import pandas as pd

dimdate = pd.DataFrame([['DATETIME_SKEY', 'TIMESTAMP', 'Y', 'No', 'NOT NULL', 1, 'Data derived from DIMDATE.DATETIME_SKEY']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])

dimdate.style.hide_index().set_caption('DIMDATE')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME_SKEY,TIMESTAMP,Y,No,NOT NULL,1,Data derived from DIMDATE.DATETIME_SKEY


## Dimension Table DIMPLATFORM

In [13]:
import pandas as pd

dimplatform = pd.DataFrame([['PLATFORM_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived DIMPLATFORM.PLATFORM_SKEY'],
                          ['PLATFORM_TYPE', 'VARCHAR2(200 BYTE)', 'N', 'No', 'NOT NULL', 2, 'Data derived DIMPLATFORM.PLATFORM']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimplatform.style.hide_index().set_caption('DIMPLATFORM')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
PLATFORM_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived DIMPLATFORM.PLATFORM_SKEY
PLATFORM_TYPE,VARCHAR2(200 BYTE),N,No,NOT NULL,2,Data derived DIMPLATFORM.PLATFORM


## Dimension Table DIMSITE


In [14]:
import pandas as pd

dimsite = pd.DataFrame([['SITE_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived DIMSITE.SITE_SKEY'],
                          ['SITE', 'VARCHAR(200)', 'N', 'No', 'NOT NULL', 2, 'Data derived DIMSITE.SITE']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimsite.style.hide_index().set_caption('DIMSITE')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
SITE_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived DIMSITE.SITE_SKEY
SITE,VARCHAR(200),N,No,NOT NULL,2,Data derived DIMSITE.SITE


## Dimension Table DIMVIDEO


In [15]:
import pandas as pd

dimvideo = pd.DataFrame([['VIDEO_SKEY', 'INTEGER', 'Y', 'No', 'NOT NULL', 1, 'Data derived from DIMVIDEO.VIDEO_SKEY'],
                          ['VIDEO_TITLE', 'TEXT', 'N', 'No', 'NOT NULL', 2, 'Data derived from DIMVIDEO.VIDEO_TITLE']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
dimvideo.style.hide_index().set_caption('DIMVIDEO')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
VIDEO_SKEY,INTEGER,Y,No,NOT NULL,1,Data derived from DIMVIDEO.VIDEO_SKEY
VIDEO_TITLE,TEXT,N,No,NOT NULL,2,Data derived from DIMVIDEO.VIDEO_TITLE


## Fact Table FACTTABLE


In [16]:
import pandas as pd
facttable = pd.DataFrame([['DATETIME_SKEY', 'TIMESTAMP', 'N', 'No', 'NOT NULL', 1, 'Data derived from FACTTABLE.DATETIME_SKEY'],
                          ['Platform_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 2, 'Data derived from FACTTABLE.PLATFORM_SKEY'],
                          ['Site_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 3, 'Data derived from FACTTABLE.SITE_SKEY'],
                          ['Video_SKEY', 'INTEGER', 'N', 'No', 'NOT NULL', 4, 'Data derived from FACTTABLE.VIDEO_SKEY'],
                          ['Events', 'VARCHAR2(150 BYTE)', 'N', 'No', 'NOT NULL', 5, 'Data derived from FACTTABLE.EVENTS']],
                  columns=['COLUMN_NAME', 'DATA_TYPE', 'PK', 'NULLABLE', 'DATA_DEFAULT', 'COLUMN_ID', 'COMMENTS'])
                  
facttable.style.hide_index().set_caption('FACTTABLE')

COLUMN_NAME,DATA_TYPE,PK,NULLABLE,DATA_DEFAULT,COLUMN_ID,COMMENTS
DATETIME_SKEY,TIMESTAMP,N,No,NOT NULL,1,Data derived from FACTTABLE.DATETIME_SKEY
Platform_SKEY,INTEGER,N,No,NOT NULL,2,Data derived from FACTTABLE.PLATFORM_SKEY
Site_SKEY,INTEGER,N,No,NOT NULL,3,Data derived from FACTTABLE.SITE_SKEY
Video_SKEY,INTEGER,N,No,NOT NULL,4,Data derived from FACTTABLE.VIDEO_SKEY
Events,VARCHAR2(150 BYTE),N,No,NOT NULL,5,Data derived from FACTTABLE.EVENTS


# **Pre-process the Raw Data**

## 1. Data auditing

In [117]:
import pandas as pd

df = pd.read_csv('source.csv', usecols=[0,1,2])
df.columns = ['DateTime', 'VideoTitle', 'events']

print(pd.Series({c: df[c].map(lambda x: len(str(x))).max() for c in df}))

DateTime       24
VideoTitle    157
events         95
dtype: int64


## 2. Identify Objects for ETL Process

In [121]:
# Identify the type of PLATFORM and SITE
import pandas as pd

df = pd.read_csv('source.csv', usecols=[0,1,2])
df.columns = ['DateTime', 'VideoTitle', 'events']
df[['0','1','2','3','4']] = df['VideoTitle'].str.split("|",expand=True,)

df['0'].unique()

array(['App Web', 'news', 'App iPhone', 'App Android', 'App iPad'],
      dtype=object)

## 3. The sql script to create the table

check <font color='#33AFFF'>snowflake.sql</font>

# **NOTE**

1. SKEY stands for surrogate key.  
<br>
2. The current design is <font color='#33AFFF'>Dimension Type One</font>.  
<br>
3. If the source dimension data contains not only the PK but also some attributes, and we want to track the changes of attributes, we should use <font color='#33AFFF'>Dimension Type Two</font>. 

**One sample of Dimension Type Two**

In [22]:
import pandas as pd
raw_table = pd.DataFrame([['P01', 'iPhone 13', '1429', 'Noel Leeming'],
                          ['P03', 'iPhone 12', '1049', 'Noel Leeming']],
                  columns=['Product_ID', 'Product', 'Price', 'Location'])

raw_table.style.hide_index().set_caption('Data from 01/03/2022')

Product_ID,Product,Price,Location
P01,iPhone 13,1429,Noel Leeming
P03,iPhone 12,1049,Noel Leeming


**Data in dimension table**

In [37]:
import pandas as pd
import numpy as np

raw_table = pd.DataFrame([['111', 'P01', 'iPhone 13', '1429', 'Noel Leeming', 'Y', '31/12/2021', '31/12/9999'],
                          ['112', 'P02', 'iPhone 13 mini', '1249', 'Noel Leeming', 'Y', '20/01/2022', '31/12/9999']],
                  columns=['Product_SKEY', 'Product_ID', 'Product', 'Price', 'Location', 'Current_Flag', 'Start_Date', 'End_Date'])

# raw_table.style.hide_index().set_caption('Data in dimension table')

,Product_SKEY,Product_ID,Product,Price,Location,Current_Flag,Start_Date,End_Date
0,111,P01,iPhone 13,1429,Noel Leeming,Y,31/12/2021,31/12/9999
1,112,P02,iPhone 13 mini,1249,Noel Leeming,Y,20/01/2022,31/12/9999
